In [ ]:
import os
import importlib

import pandas as pd
import numpy as np
import geopy
from geopy import distance
from nile.api.v1 import filters as nf
from nile.api.v1 import extractors as ne
from nile.api.v1 import aggregators as na
from qb2.api.v1 import filters as qf

from projects.common import geo
from projects.common.nile import dates, test_utils
from projects.cpo_model import project_config, default_values, common

In [ ]:
pd.options.display.max_rows = 300
pd.options.display.max_columns = 100

In [ ]:
_ = importlib.reload(default_values)
_ = importlib.reload(project_config)
_ = importlib.reload(common)
_ = importlib.reload(test_utils)
_ = importlib.reload(geo)

#### Declare non-constant objects

In [ ]:
cluster = project_config.get_project_cluster(
    parallel_operations_limit=5,
    pool='taxi-delivery',
    extra_requirements=['numpy'],
)

#### Declare constants

In [ ]:
DATE_START = '2020-01-01'
DATE_END = '2021-03-18'

### Get data from YT

In [ ]:
job = cluster.job()

In [ ]:
# Source tables.
market_deliveries = job.table(
    default_values.YT.MARKET_DELIVERIES
)

eda_places = job.table(
    default_values.YT.EDA_PLACES
)

eda_dm_order = job.table(
    default_values.YT.EDA_DM_ORDER.format(
        dates.range_selector(DATE_START, DATE_END, '%Y-%m')
    )
)

In [ ]:
# # Geocoding.
# no_gps_final_point = (
#     market_deliveries
#     .filter(
#         nf.not_(qf.defined('final_delivery_point_gps')),
#         qf.defined('final_delivery_point_address')
#     )
#     .unique('final_delivery_point_address')
#     .project('final_delivery_point_address', point_type=ne.const('final'))
# )

# outlet_address = (
#     market_deliveries
#     .filter(
#         qf.defined('outlet_address')
#     )
#     .unique('outlet_address')
#     .project('outlet_address', point_type=ne.const('outlet'))
# )

# dropship_address = (
#     market_deliveries
#     .filter(
#         qf.defined('dropship_address')
#     )
#     .unique('dropship_address')
#     .project('dropship_address', point_type=ne.const('dropship'))
# )

# fulfillment_address = (
#     market_deliveries
#     .filter(
#         qf.defined('fulfillment_address')
#     )
#     .unique('fulfillment_address')
#     .project('fulfillment_address', point_type=ne.const('fulfillment'))
# )

# external_sorting_center_address = (
#     market_deliveries
#     .filter(
#         qf.defined('external_sorting_center_address')
#     )
#     .unique('external_sorting_center_address')
#     .project('external_sorting_center_address', point_type=ne.const('external_sorting_center'))
# )

# internal_sorting_center_address = (
#     market_deliveries
#     .filter(
#         qf.defined('internal_sorting_center_address')
#     )
#     .unique('internal_sorting_center_address')
#     .project('internal_sorting_center_address', point_type=ne.const('internal_sorting_center'))
# )

# job.concat(*[
#     no_gps_final_point, outlet_address, dropship_address,
#     fulfillment_address, external_sorting_center_address,
#     internal_sorting_center_address
# ]).put(os.path.join(default_values.YT.PROJECT_FOLDER, 'no_gps_points'))

In [ ]:
# Filter out Moscow.
# moscow_deliveries = (
#     market_deliveries
#     .filter(
#         nf.equals('delivery_region_id', 213),
#     )
#     .put(os.path.join(default_values.YT.PROJECT_FOLDER, 'moscow_deliveries_2020'))
# )

# # Filter out regions.
# regional_deliveries = (
#     market_deliveries
#     .filter(
#         nf.not_(nf.equals('delivery_region_id', 1)),
#         nf.not_(nf.equals('delivery_region_id', 213)),
#         nf.not_(nf.equals('delivery_region_id', 2)),
#         nf.not_(nf.equals('delivery_region_id', 10174)),
#     )
#     .put(os.path.join(default_values.YT.PROJECT_FOLDER, 'regional_deliveries_2020'))
# )

# # Filter out Kazan
# kazan_deliveries = (
#     market_deliveries
#     .filter(
#         nf.equals('delivery_region_id', 43),
#     )
#     .put(os.path.join(default_values.YT.PROJECT_FOLDER, 'kazan_deliveries_2020'))
# )

# novosib_deliveries = (
#     market_deliveries
#     .filter(
#         nf.equals('delivery_region_id', 65),
#     )
#     .put(os.path.join(default_values.YT.PROJECT_FOLDER, 'novosibirsk_deliveries_2020'))
# )

# # Filter out Lavka addresses.
lavka_places = (
    eda_places
    .filter(
        nf.equals('name', test_utils.to_bytes('Яндекс.Лавка')),
#         nf.equals('enabled', 1),
    )
    .project('location_lat', 'location_lon', 'name', 'address_city',
             'address_full', 'couriers_type', 'disable_details_status',
             'utc_disable_details_available_dttm')
    .put(os.path.join(default_values.YT.PROJECT_FOLDER, 'lavka_places'))
)

# # Filter out Moscow Lavka addresses.
lavka_places_moscow = (
    lavka_places
    .filter(nf.custom(lambda lat, lon: 55 < lat < 56.1 and 37 < lon < 38.3,
                      'location_lat', 'location_lon'))
    .put(os.path.join(default_values.YT.PROJECT_FOLDER, 'lavka_places_moscow'))
)

# lavka_orders = (
#     eda_dm_order
#     .filter(
#         qf.defined('place_lat', 'place_lon', 'location_lat', 'location_lon',),
#         nf.equals('brand_store_flg', True),
#         nf.equals('order_status', 4),
#     )
#     .project(
#         'brand_store_flg', 'courier_type', 'drop_off_dur_sec', 'in_place_dur_sec',
#         'local_accepted_dttm', 'local_arrival_to_customer_fact_dttm',
#         'local_arrival_to_place_fact_dttm', 'local_courier_assigned_dttm',
#         'local_created_dttm', 'local_given_dttm', 'local_ready_dttm',
#         'location_lat', 'location_lon', 'order_items_cost_lcy', 'order_status',
#         'time_to_delivery', 'to_customer_dur_sec', 'to_place_dur_sec',
#         'user_cost_w_vat_lcy', 'distance_to_customer', 'distance_to_place', 'place_lat', 'place_lon',
#         distance_to_customer_sl=ne.custom(
#             lambda plat, plon, llat, llon:
#             geo.earth_distance(llat, llon, plat, plon),
#             'place_lat', 'place_lon', 'location_lat', 'location_lon',
#         ),
#     )
#     .put(os.path.join(default_values.YT.PROJECT_FOLDER, 'lavka_orders'))
# )

# lavka_orders_moscow = (
#     lavka_orders
#     .filter(
#         nf.custom(lambda lat, lon: 55 < lat < 56.1 and 37 < lon < 38.3,
#                   'location_lat', 'location_lon')
#     )
#     .put(os.path.join(default_values.YT.PROJECT_FOLDER, 'lavka_orders_moscow'))
# )

In [ ]:
job.flow_graph

In [ ]:
# job.run()

### Get market orders for 1M+ cities from 2020-11-01 to 2021-01-10

In [ ]:
CITY_MAP = {
    'Москва': 213,
    'Санкт-Петербург': 2,
    'Новосибирск': 65,
    'Екатеринбург': 54,
    'Нижний Новгород': 47,
    'Казань': 43,
    'Челябинск': 56,
    'Омск': 66,
    'Самара': 51,
    'Ростов-на-Дону': 39,
    'Уфа': 172,
    'Красноярск': 62,
    'Пермь': 50,
    'Воронеж': 193,
    'Волгоград': 38,
    'Краснодар': 35,
}

In [ ]:
CITY_MAP.values()

In [ ]:
job = cluster.job()

In [ ]:
market_deliveries = job.table(
    default_values.YT.MARKET_DELIVERIES
)

In [ ]:
(
    market_deliveries
    .filter(
        nf.custom(lambda x: '2020-11-01' <= test_utils.to_string(x)[:10] <= '2021-01-10'
                  if x else False,
                  'original_delivery_to_datetime'),
        nf.custom(lambda x: test_utils.to_string(x) in CITY_MAP.values(),
                  'delivery_region_id')
    )
    .put('//home/taxi-delivery/analytics/dev/cpo_model/m+_city_orders_2020-11-01_2021-01-10')
)

In [ ]:
job.flow_graph

In [ ]:
job.run()

### Filter Moscow orders for 2020-10-29 and 2020-10-30

In [ ]:
job = cluster.job()

In [ ]:
# (
#     job
#     .table('//home/taxi-delivery/analytics/dev/cpo_model/lavka_orders_moscow')
#     .filter(
#         nf.custom(lambda x: '2020-10-29 00:00:00' <= test_utils.to_string(x) <= '2020-11-04 00:00:00'
#                   if x else False,
#                   'local_created_dttm')
#     )
#     .put('//home/taxi-delivery/analytics/dev/cpo_model/lavka_orders_moscow_2020-10-29_2020-11-04')
# )

In [ ]:
(
    job
    .table('//home/taxi-delivery/analytics/dev/cpo_model/moscow_deliveries_2020')
    .filter(
        nf.custom(lambda x: '2020-11-02 00:00:00' <= test_utils.to_string(x) < '2020-11-09 00:00:00'
                  if x else False,
                  'original_delivery_to_datetime')
    )
    .put('//home/taxi-delivery/analytics/dev/cpo_model/moscow_deliveries_2020-10-29_2020-11-04')
)

(
    job
    .table('//home/taxi-delivery/analytics/dev/cpo_model/kazan_deliveries_2020')
    .filter(
        nf.custom(lambda x: '2020-11-02 00:00:00' <= test_utils.to_string(x) < '2020-11-09 00:00:00'
                  if x else False,
                  'original_delivery_to_datetime')
    )
    .put('//home/taxi-delivery/analytics/dev/cpo_model/kazan_deliveries_2020-10-29_2020-11-04')
)

# (
#     novosib_deliveries
#     .filter(
#         nf.custom(lambda x: '2020-11-02 00:00:00' <= test_utils.to_string(x) < '2020-11-09 00:00:00'
#                   if x else False,
#                   'original_delivery_to_datetime')
#     )
#     .put('//home/taxi-delivery/analytics/dev/cpo_model/novosibirsk_deliveries_2020-10-29_2020-11-04')
# )

# (
#     regional_deliveries
#     .filter(
#         nf.custom(lambda x: '2020-11-02 00:00:00' <= test_utils.to_string(x) < '2020-11-09 00:00:00'
#                   if x else False,
#                   'original_delivery_to_datetime')
#     )
#     .put('//home/taxi-delivery/analytics/dev/cpo_model/regional_deliveries_2020-10-29_2020-11-04')
# )

In [ ]:
job.flow_graph

In [ ]:
job.run()

### Get lavka couriers stats

In [ ]:
df = test_utils.decode_dataframe(
    cluster
    .read('//home/taxi-delivery/analytics/dev/cpo_model/lavka_orders_moscow')
    .as_dataframe()
)

In [ ]:
df.head()

### Calculate average courier speed by type

In [ ]:
def mps_to_kph(mps):
    return mps * 3.6

def kph_to_mps(kph):
    return kph / 3.6

COURIER_TYPE_TO_MAX_SPEED_MPS = {
    'bicycle': kph_to_mps(40),
    'electric_bicycle': kph_to_mps(40),
    'vehicle': kph_to_mps(120),
    'motorcycle': kph_to_mps(120),
    'pedestrian': kph_to_mps(30),
}
SPEED_TYPES = {'mps', 'kph'}



def speed_is_valid(speed, courier_type, speed_type='mps'):
    assert speed_type in SPEED_TYPES
    
    if speed_type == 'kph':
        speed = kph_to_mps(speed)
    
    if speed <= 0 or pd.isnull(speed):
        return False
    
    max_speed = COURIER_TYPE_TO_MAX_SPEED_MPS.get(courier_type)
    if not max_speed:
        return False
    
    if speed >= max_speed:
        return False
    
    return True

In [ ]:
df['courier_speed_mps'] = df['distance_to_customer_sl'] / df['to_customer_dur_sec']

In [ ]:
df['valid_speed_flg'] = np.vectorize(speed_is_valid)(df.courier_speed_mps, df.courier_type, 'mps')

In [ ]:
df[df['valid_speed_flg']]['courier_speed_mps'].hist(bins=100)

In [ ]:
(
    df[df['valid_speed_flg']][['courier_type', 'courier_speed_mps']]
    .dropna()
    .groupby('courier_type')
    .agg(
        {'courier_speed_mps': ['mean', 'min', 'max']}
    )
)

### Calculate average real distance to straight-line distance

In [ ]:
df['real_dist_to_sl'] = (
    df['distance_to_customer'] / df['distance_to_customer_sl']
)

In [ ]:
df[
    df['real_dist_to_sl']!=np.inf
].groupby('courier_type').mean()['real_dist_to_sl']